# Data input

Submission deadline: Sept 20 11:59 AM

In general data can be acquired as:
* downloadable files (e.g. CSV, txt, Excel, JSON files and other common data formats)
* through API (Application Program Interface)
* through databases 

In this lab we will provide examples how to work with downloadable data in the two most common formats:
* comma-separated-values (CSV) or text files (txt) more generally 
* and Excel or XLS files

Also as urban analytics is often inherently spatial we'll consider another specific format often used for the spatial data: JSON

We will also provide some basic descriptive analytics, like getting the number of records, defining timeframes or summing up numeric columns.

It is always a good idea to start data analytics with such descriptive analysis as not only it gives an idea of the scale (and thus utility) of the data, but also allows to spot possible issues

In [ ]:
import pandas as pd #first we'll need module pandas to work with the dataframes
%matplotlib inline

## Example 1. The 311 service requests (CSV) in Boston

The data contains all 311 call service requests, including their time, location, type of issue and many other details

Source https://data.boston.gov/dataset/311-service-requests

In [ ]:
#lets specify the file location on the web
#we can actually upload the data directly from there!
#the data is provided by year, so consider the most recent full year available - 2019
fname = '311_service_requests_2020.csv'

In [ ]:
boston311 = pd.read_csv(fname) #upload the data

In [ ]:
#preview the data
boston311.head()

In [ ]:
#get the list of columns
boston311.columns

In [ ]:
#get the total number of records
len(boston311)

In [ ]:
#summary statistics for all the numeric columns
boston311.describe()

Gives a good idea of how many records have a valid number defined and also averages and ranges for latitute/logitude

statistics for zipcode and id won't make that much sense 

as while these data has numeric type its categorical in nature and computing averages is somewhat meaningless

Some additional descriptive analysis one might be interested is to learn more about the categorical variables, e.g. subject, reason, type, neighborhood, zip_code

In [ ]:
#first let's get the list of unique values for each, or at least the number of such values

In [ ]:
boston311.subject.unique()

In [ ]:
len(boston311.subject.unique())

In [ ]:
len(boston311.reason.unique())

In [ ]:
len(boston311.type.unique())

And if e.g. we want to know the number of complaints per subject we can group the dataframe as following

In [ ]:
boston311['subject'].value_counts()

In [ ]:
#or visualizing as a histogram
boston311[['subject','case_enquiry_id']].groupby(by=['subject']).count().plot.bar()

### Task 1. Count the number of unique neighborhoods (1 pt)

In [ ]:
# complete the below function with solution code. The function should return the number of unique neighborhoods

def printCount(boston311):
    
    '''Count the number of unique neighborhoods'''
    
    # YOUR CODE HERE
    raise NotImplementedError()

### Task 2: Visualize the number of complaints by neighborhood (1 pt)

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

## Example 2. Real-estate data sales (XLS)

The data contains information on all the residential real estate sold in NYC, including address of the house/unit, its size, age, time of sale and of course the price

In [ ]:
url = '2015_brooklyn.xls'

In [ ]:
re_sales = pd.read_excel(url)

In [ ]:
re_sales.head()

We'll need to skip first 4 lines to make it pandas-frienly 

And also consider alternative data ingestion approach: download to the local folder first and then upload from local drive

In [ ]:
#now upload the data, also skipping first 4 rows
re_sales = pd.read_excel(dataDir+fname,skiprows=4)

In [ ]:
re_sales.head() #now the dataframe looks reasonable

In [ ]:
re_sales.columns #list of the columns to assess the metadata

In [ ]:
#notice that field names contain an artifact from the original data represenation - end of line symbol '\n'
#this way we won't be able to access the fields as before using "." syntax, but we can still use "[]" syntax like

In [ ]:
re_sales['SALE PRICE\n'].sum() #this will be the total sales volume

In [ ]:
re_sales.describe() #statistics for numeric columns

It also helps us spot some issues with the data - zero min's for zip code, size, year (also average age of 1774 does not look right and is likely affected by that), sale price show that some records are unreliable

Also lets introduce some descriptive analysis, e.g. total price and average price per unit

In [ ]:
re_sales['SALE PRICE\n'].mean() #average price per unit

In [ ]:
#or visualizing an average sale price per zip code 
#ideally this would make more sense as a spatial visualization on the map. but let us get to that later
#we'll use groupby here. Note that groupby will apply mean to all the fields other than the ones used for grouping (by)
#so we first filter the datarame to keep only two columns - 'ZIP CODE\n','SALE PRICE\n' - first for grouping, second for taking averages
re_sales[['ZIP CODE\n','SALE PRICE\n']].groupby(by=['ZIP CODE\n']).mean()

In [ ]:
re_sales[['ZIP CODE\n','SALE PRICE\n']].groupby(by=['ZIP CODE\n']).mean().plot.bar()

But price per unit is not always the best characteristics of the market as units might be quite different, ranging from small studios to large houses. Besides particularly expensive ones may affect the averages. Median price is often used along with the price per square foot. In the next excercise, we'll try to compute an average price per square foot. For now let's skip the technical issue like missing/zero data and just plow ahead. In the next lab we'll talk about how those should addressed.

### Task 3.  (1 pt)
Quantify the average price per square foot per zip code. Visualize result as a bar plot. Which zip codes are the three most expensive ones?

Hints: First, use "group_by" to create an aggregate with cumulative price and sq foot size and then add a new field through assignment dividing one by the other. 

You can use sort_values to order dataframe by price per zip code and see the top three zip codes.

Not mandatory, but if you want python to give you a specific list of the top three zip codes, you can use .iloc to slice it and get the index converting it to a list. But then make sure to ignore zip codes with infinite prices (likely due to zero house sizes).

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

## Example 3. Street Pavement Rating

The New York City Department of Transportation is responsible for keeping the City's streets in good repair. The Agency performs ongoing assessment of New York City streets. Ratings are based on a scale from 1 to 10, and results are grouped in the following categories: Good (%) - ratings of 8 to 10, Fair (%) - ratings of 4 to 7, and Poor (%) - ratings of 1 to 3. Rating dates started from 1994 to 2019. This data will allow you to create a map.

GeoJSON is a JSON based format designed to represent the geographical features with their non-spatial attributes. Besides GeoJSON, shapefile is also a common format for geographical datasets. For more information, please check https://geojson.org/.

As JSON contains spatial information (in this case shapes of the streets) along with the data, we'll need a slightly different vehicle to load it. There is a module geopandas which we'll use below. In short its just like pandas, but having a special column "geometry" which is meant to contain spatial info

In [ ]:
import geopandas as gpd
url = 'Street_Pavement_Rating.geojson'

rating = gpd.read_file(url)

In [ ]:
rating.shape #shape of the dataframe - first is the number of records (streets)

In [ ]:
rating.head() #lets preview the data

In [ ]:
rating.iloc[:,:-1].describe() #note that we can't apply describe to a geometry column so need to exclude it first

In [ ]:
#Notice that this time describe look differently. This is because the values in the dataframe are not treated as numbers but categorical values

In [ ]:
#if we want them as numbers we need to perform type convertion
rating['length'] = rating['length'].astype(int)
rating['length'].describe()

In [ ]:
#now let's count how many streets are in either condition based on 'rating_word'
rating['rating_word'].value_counts()

In [ ]:
#now let's count the cumulative length of those in either condition based on 'rating_word'
rating[['rating_word','length']].groupby(by=['rating_word']).sum()

In [ ]:
#and visualize overall conditions of the roads using piechart
rating[['rating_word','length']].groupby(by=['rating_word']).sum().plot.pie(y='length')

And let's use the true power of geopandas - visualize the streetmap

In [ ]:
rating.plot(column=None, figsize=(10,10)) #first just the shapes of the streetmap without any information

In [ ]:
#and now visualize the streets according to their status 
rating.plot(column='rating_word',cmap='Spectral',figsize=(10,10),legend=True)

### Task 4. (1 pt)
Visualize only the poorly maintained roads so we can zoom into those that need particular attention.

HINT: Filter the dataframe first using slicing by an appropriate condition on rating.rating_word

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

### Task 5. COVID-19 Data by ZIP Code (3 pts)

This dataset includes confirmed case rates, case counts, death rates, death counts, and the percent of people tested with a positive result since March. Follwing download links refer to the dataset uploaded on Sep 1st. Dataset is provided by NYC health, please check https://www1.nyc.gov/site/doh/covid/covid-19-data.page for more information and other COVID19 related datasets.

Task: 
    
    a) Aggregate dataset by Borough, calculate total cases amount in each Borough, and then visualize as a barplot
    
    b) Calculate borough-wise positive case percentage among all the tests administered and 
    
    c) among the borough population (%% of populating tested positive), and then visualize as barplots. 
    
    Notice that both quantities b) and c) matter as otherwise the percentage of people tested positive might depend on the testing availability and practice which are not necessary consistent among boroughs.


Hint: The column for case amount is “COVID_CASE_COUNT". Try groupby function in pandas to get total cases and tests by borough. Positive case percentage is ‘100*COVID_CASE_COUNT'/'TOTAL_COVID_TESTS'. To calculate positive case percentage, you might first need a borough level dataframe.

In [ ]:
url = 'https://raw.githubusercontent.com/nychealth/coronavirus-data/7953c97d1e58bbed9934ea04affb12ca74d9c0fb/data-by-modzcta.csv'

df = pd.read_csv(url)

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()